# Applied Data Science Capstone, Week 2 Assignment - Part 1

In [1]:
# I'm using Venv, check if the kernel is correct

!which python

/Users/qzhou/PycharmProjects/foursquare/bin/python


## scrape Wikipedia to get neighborhood informantion of Toronto

[List of neighborhood of Toronto](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)

In [2]:
# find the table with BeautifulSoup. 

from bs4 import BeautifulSoup
import requests

scrape_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(scrape_url)
soup = BeautifulSoup(html.content, 'html5lib')
tables = soup.findAll('table', {'class': 'wikitable'})[0]

```
note that, header is enclosed in <tbody><tr> <td></td> </tr>, and
table rows are enclosed in <tr> <td></td> </tr>
```

In [3]:
# extract headers

header = []
for item in tables.find('tr').findAll('th'):
    header.append(item.text.strip())

# extract rows

tbl = []
for row in tables.findAll('tr'):
    r = []
    items = row.findAll('td')
    for item in items:
        r.append(item.text.strip())
    tbl.append(r)

In [4]:
# load table as pd.DataFrame

import pandas as pd

neighborhood = pd.DataFrame(data=tbl, columns=header)
neighborhood.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


- Only process the cells that have an assigned borough. Ignore cells with a __borough__ that is __Not assigned__.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that __M5A__ is listed twice and has two neighborhoods: __Harbourfront__ and __Regent Park__. These two rows will be __combined into one row with the neighborhoods separated with a comma__ as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the __neighborhood will be the same as the borough__. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [5]:
# clean data

# Borough Not assigned = pd.NaT
neighborhood.replace(to_replace={'Borough':'Not assigned'}, value={'Borough': pd.NaT}, inplace=True)
neighborhood.dropna(axis=0, inplace=True)

# Neighbourhood Not assigned = Borough
condition = neighborhood['Neighbourhood'] == 'Not assigned'
neighborhood.loc[condition, 'Neighbourhood'] = neighborhood.loc[condition, 'Borough']

In [6]:
# merge neighbourhood if Postcode is the same.

def mergePostcode(x):
    return pd.Series(
        dict(Borough = x['Borough'].value_counts().index[0], 
             Neighbourhood = "%s" % ', '.join(x['Neighbourhood']))
    )

df = neighborhood.groupby('Postcode').apply(mergePostcode)
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


__df__ stores the cleaned information about neighbourhood of Toronto.

In [7]:
df.shape

(103, 3)